In [14]:
from helpers.apps.json_logic import apply_json_logic
from helpers.serializers import get_obj_serialized
from helpers.strings import get_obj_from_path, to_flatten_str
from helpers.apps.daily_reports import generate_exported_file
import re
import pandas as pd
from tqdm.notebook import tqdm
from django.db.models import Q

In [15]:
def ordenar_chave(elemento):
    partes = re.split(r'(\d+)', elemento)
    return (partes[0], int(partes[1]))

In [16]:
def get_exporter_extra_columns(company):
    return get_obj_from_path(
        company.custom_options, "reporting__exporter__extra_columns"
    )


In [17]:
def get_uf(uf_code):
    if type(uf_code) != 'str':
        uf_code=str(uf_code)
    
    uf=[{'id': '12', 'name': 'AC'}, {'id': '27', 'name': 'AL' }, {'id': '13', 'name': 'AM'}, {'id': '16', 'name': 'AP'}, 
        {'id': '29', 'name': 'BA'}, {'id': '23', 'name': 'CE' }, {'id': '53', 'name': 'DF'}, {'id': '32', 'name': 'ES'}, 
        {'id': '52', 'name': 'GO'}, {'id': '21', 'name': 'MA' }, {'id': '31', 'name': 'MG'}, {'id': '50', 'name': 'MS'}, 
        {'id': '51', 'name': 'MT'}, {'id': '15', 'name': 'PA' }, {'id': '25', 'name': 'PB'}, {'id': '26', 'name': 'PE'}, 
        {'id': '22', 'name': 'PI'}, {'id': '41', 'name': 'PR' }, {'id': '33', 'name': 'RJ'}, {'id': '24', 'name': 'RN'},
        {'id': '11', 'name': 'RO'}, {'id': '14', 'name': 'RR' }, {'id': '43', 'name': 'RS'}, {'id': '42', 'name': 'SC'},
        {'id': '28', 'name': 'SE'}, {'id': '35', 'name': 'SP' }, {'id': '17', 'name': 'TO'}]
    uf_translation={item['id']: item['name'] for item in uf}
#     print(uf_translation)    
    
    return uf_translation[uf_code]

In [18]:
def get_exporter_extra_columns_parsed_infos(exporter_extra_columns):
    extra_columns = {}
    for extra_column in exporter_extra_columns:
        column_key = extra_column.get("key", "")
        extra_columns[column_key] = extra_column.get("header", "")
    return extra_columns

In [19]:
def get_values_from_reporting_extra_columns(record, extra_columns):
    new_val = {}
    if extra_columns:
        reporting_formatted = get_obj_serialized(record, is_occurrence_record=True)
#         new_val[("Data do registro")] = getattr(record,'datetime','')
#         new_val[("Data de criação")] = getattr(record,'created_at','')
#         new_val[("Atualizado em")] = getattr(record,'updated_at','')
#         new_val[("Nº Registro")] = getattr(record,'number','')
#         new_val[("Unidade")] = getattr(getattr(record,'company',''),'name','')
#         new_val[("Status")] = getattr(getattr(record,'approval_step',''),'name','')
#         new_val[("Profissional")] = record.user.get_full_name() if getattr(record,'user','') != '' else ''
#         new_val[("Função")] = getattr(getattr(record,'user',''),'metadata','').get('role','')
#         new_val[("Nº Registro Conselho")] = ''
#         new_val[("Equipe")] = getattr(getattr(record,'firm',''),'name','')
#         new_val[("UF")] = get_uf(getattr(record,'uf_code',''))
#         new_val[("Município")] = getattr(getattr(record,'city',''),'name','')
#         new_val[("Localidade")] = getattr(getattr(record,'location',''),'name','')
#         new_val[("Local")] = getattr(record,'place_on_dam','')
#         new_val[("Corpo Hídrico")] = record.end_km
#         new_val[("KM da Occorrência")] = record.occurrence_type.name
#         new_val[("Outra Referência")] = record.occurrence_type.name
        
        for item in extra_columns:
            json_logic = None
            key = item.get("key", "")
            logic = item.get("logic", False)
            header = item.get("header", "")
            is_date = item.get("isDate", False)
            if key and logic:
                try:
                    json_logic = apply_json_logic(logic, reporting_formatted)
                except Exception:
                    pass
            if not json_logic:
                try:
                    json_logic = reporting_formatted.form_data[key]
                except Exception:
                    pass
            if is_date and json_logic:
                json_logic = format_date(json_logic)
            new_val[(header)] = json_logic
    return new_val

In [20]:
Company.objects.all().count()

47

In [21]:
# company=Company.objects.get(name__icontains='DER/SP')
companies  = list(Company.objects.filter(
    name__icontains='GSO').exclude(name='Energia Compartilhado GSO').order_by('name').values_list('name',flat=True))
companies

['GSO - PCH-JOSÉ GELÁZIO DA ROCHA',
 'GSO - PCH-RONDONÓPOLIS',
 'GSO - UHE-CANA BRAVA',
 'GSO - UHE-ESTREITO',
 'GSO - UHE-ITA',
 'GSO - UHE-JAGUARA',
 'GSO - UHE-MACHADINHO',
 'GSO - UHE-MIRANDA',
 'GSO - UHE-PASSO FUNDO',
 'GSO - UHE-PONTE DE PEDRA',
 'GSO - UHE-SALTO OSORIO',
 'GSO - UHE-SALTO SANTIAGO',
 'GSO - UHE-SÃO SALVADOR',
 'GSO - UTPS']

In [22]:
counter=0
for name in tqdm(companies):
    ocrs=OccurrenceRecord.objects.filter(Q(place_on_dam='ZAS') | Q(place_on_dam='ZSS'),
        company__name=name
    ).prefetch_related('company','occurrence_type').order_by('number','occurrence_type')
    print(name,ocrs.count())
    counter+=ocrs.count()
print('Total:',counter)

  0%|          | 0/14 [00:00<?, ?it/s]

GSO - PCH-JOSÉ GELÁZIO DA ROCHA 1
GSO - PCH-RONDONÓPOLIS 0
GSO - UHE-CANA BRAVA 36
GSO - UHE-ESTREITO 3793
GSO - UHE-ITA 744
GSO - UHE-JAGUARA 453
GSO - UHE-MACHADINHO 655
GSO - UHE-MIRANDA 337
GSO - UHE-PASSO FUNDO 62
GSO - UHE-PONTE DE PEDRA 60
GSO - UHE-SALTO OSORIO 241
GSO - UHE-SALTO SANTIAGO 794
GSO - UHE-SÃO SALVADOR 948
GSO - UTPS 15
Total: 8139


In [23]:
dados=[]
for name in tqdm(companies):
    ocrs=OccurrenceRecord.objects.filter(
#         Q(place_on_dam='ZAS') | Q(place_on_dam='ZSS'),
        company__name=name
    ).prefetch_related('company','occurrence_type').order_by('number','occurrence_type')
    print(name)
    
    for record in tqdm(ocrs):
        new_val = {}
        
        new_val[("Data do registro")] = getattr(record,'datetime','').strftime('%d/%m/%Y %H:%M:%S')
        new_val[("Data de criação")] = getattr(record,'created_at','').strftime('%d/%m/%Y %H:%M:%S')
        new_val[("Atualizado em")] = getattr(record,'updated_at','').strftime('%d/%m/%Y %H:%M:%S')
        new_val[("Nº Registro")] = record.number
        new_val["Unidade"] = name
        new_val[("Classe")] = record.occurrence_type.name
        new_val[("Status")] = getattr(getattr(record,'approval_step',''),'name','')
        new_val[("Profissional")] = record.user.get_full_name() if getattr(record,'user','') != '' else ''
        new_val[("Função")] = getattr(getattr(record,'user',''),'metadata','').get('role','') if getattr(getattr(record,'user',''),'metadata','') != '' else ''
        new_val[("Nº Registro Conselho")] = getattr(getattr(record,'user',''),'metadata','').get('board_registration','') if getattr(getattr(record,'user',''),'metadata','') != '' else ''
        new_val[("Equipe")] = getattr(getattr(record,'firm',''),'name','')
        new_val[("UF")] = get_uf(getattr(record,'uf_code',''))
        new_val[("Município")] = getattr(getattr(record,'city',''),'name','')
        new_val[("Localidade")] = getattr(getattr(record,'location',''),'name','')
        new_val[("Local")] = record.place_on_dam
        new_val[("Corpo Hídrico")] = ''
        new_val[("KM da Occorrência")] = 0
        new_val[("Outra Referência")] = record.other_reference
        if record.occurrence_type.form_fields.get('measurementColumns'):
            report=get_values_from_reporting_extra_columns(record,record.occurrence_type.form_fields['measurementColumns'])
        if record.occurrence_type.form_fields.get('measurement_columns'):
            report=get_values_from_reporting_extra_columns(record,record.occurrence_type.form_fields['measurement_columns'])
        new_val.update(report)
        new_val["Unidade"] = name
        dados.append(new_val)
        

  0%|          | 0/14 [00:00<?, ?it/s]

GSO - PCH-JOSÉ GELÁZIO DA ROCHA


  0%|          | 0/3 [00:00<?, ?it/s]

GSO - PCH-RONDONÓPOLIS


  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/1560 [00:00<?, ?it/s]

GSO - UHE-SÃO SALVADOR


  0%|          | 0/1268 [00:00<?, ?it/s]

GSO - UTPS


  0%|          | 0/16 [00:00<?, ?it/s]

In [25]:
df = pd.DataFrame.from_dict(dados)

In [26]:
df

,Data do registro,Data de criação,Atualizado em,Nº Registro,Unidade,Classe,Status,Profissional,Função,Nº Registro Conselho,...,Negociação - Data da tentativa 6,Negociação - Data 1,Negociação - Data 2,Negociação - Data 3,Negociação - Data 4,Negociação - Data 5,Negociação - Data 6,Houve acordo de compromisso de locação?,Data de assinatura do termo,Comentários sobre o acordo
0,20/06/2022 13:01:36,05/04/2023 17:51:02,05/04/2023 17:51:02,PHJG-RG-2022.0007,GSO - PCH-JOSÉ GELÁZIO DA ROCHA,Edificação de uso coletivo,Aguardando validação DC,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25/07/2023 14:14:23,25/07/2023 14:14:27,13/03/2024 13:22:19,PHJG-RG-2023.0001,GSO - PCH-JOSÉ GELÁZIO DA ROCHA,Sinalização da rota de fuga,Aceito,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25/07/2023 18:54:21,25/07/2023 18:54:24,23/10/2023 18:38:32,PHJG-RG-2023.0002,GSO - PCH-JOSÉ GELÁZIO DA ROCHA,Ponto de encontro,Aceito,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20/06/2022 13:54:38,05/04/2023 13:21:14,05/04/2023 13:21:14,PHRO-RG-2022.0004,GSO - PCH-RONDONÓPOLIS,Edificação de uso coletivo,Inativado,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20/06/2022 14:30:23,05/04/2023 13:21:14,05/04/2023 13:21:14,PHRO-RG-2022.0005,GSO - PCH-RONDONÓPOLIS,Edificação sociofamiliar,Inativado,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,25/07/2023 20:11:09,25/07/2023 20:13:16,13/03/2024 13:23:37,UTPS-RG-2023.0008,GSO - UTPS,Sinalização da rota de fuga,Aceito,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11738,25/07/2023 20:11:09,25/07/2023 20:13:18,13/03/2024 13:22:15,UTPS-RG-2023.0009,GSO - UTPS,Sinalização da rota de fuga,Aceito,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11739,25/07/2023 20:11:09,25/07/2023 20:13:19,13/03/2024 13:22:18,UTPS-RG-2023.0010,GSO - UTPS,Sinalização da rota de fuga,Aceito,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11740,25/07/2023 20:11:09,25/07/2023 20:13:21,13/03/2024 13:23:36,UTPS-RG-2023.0011,GSO - UTPS,Sinalização da rota de fuga,Aceito,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df.to_excel('Rel_Usinas.xlsx',index=False)